In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [3]:
train.shape

(250, 302)

In [4]:
train[train['target']==1.0].shape[0] / train.shape[0]

0.64

In [5]:
# y = train['target']

In [6]:
train.drop(['id'],inplace=True,axis=1)
test.drop(['id'],inplace=True,axis=1)

In [7]:
from sklearn.model_selection import train_test_split,KFold, cross_val_score
from sklearn.linear_model import LogisticRegressionCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.feature_selection import RFECV,RFE

In [8]:
import h2o
print(h2o.__version__)
from h2o.automl import H2OAutoML

h2o.init(max_mem_size='16G')

3.24.0.5
Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_212"; OpenJDK Runtime Environment (build 1.8.0_212-8u212-b03-2~deb9u1-b03); OpenJDK 64-Bit Server VM (build 25.212-b03, mixed mode)
  Starting server from /opt/conda/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp96ti0g6h
  JVM stdout: /tmp/tmp96ti0g6h/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp96ti0g6h/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.5
H2O cluster version age:,1 month and 6 days
H2O cluster name:,H2O_from_python_unknownUser_48v7qp
H2O cluster total nodes:,1
H2O cluster free memory:,14.22 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"accepting new members, healthy"


In [9]:
train_D = h2o.H2OFrame(train)
test_D = h2o.H2OFrame(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [10]:
cat = list(train.select_dtypes(['object']).columns)
cat

[]

In [11]:
x = train_D.columns
y = 'target'
# For binary classification, response should be a factor
train_D[y] = train_D[y].asfactor()

In [12]:
aml = H2OAutoML(max_models=50, seed=25, max_runtime_secs=60*10)
aml.train(x=x, y=y, training_frame=train_D)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [13]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)



model_id,auc,logloss,mean_per_class_error,rmse,mse
XGBoost_grid_1_AutoML_20190725_053020_model_10,0.733403,0.575085,0.413194,0.444003,0.197139
GLM_grid_1_AutoML_20190725_053020_model_1,0.726389,0.576526,0.427431,0.445664,0.198617
XGBoost_grid_1_AutoML_20190725_053020_model_9,0.715694,0.58059,0.345139,0.444336,0.197434
StackedEnsemble_BestOfFamily_AutoML_20190725_053020,0.713264,0.59449,0.421875,0.452472,0.204731
DeepLearning_grid_1_AutoML_20190725_053020_model_2,0.706285,4.29487,0.392361,0.53879,0.290294
XGBoost_grid_1_AutoML_20190725_053020_model_3,0.705625,0.587155,0.364583,0.447035,0.19984
XGBoost_grid_1_AutoML_20190725_053020_model_8,0.705,0.58749,0.332292,0.448142,0.200831
XGBoost_1_AutoML_20190725_053020,0.7025,0.590869,0.417361,0.449443,0.201999
StackedEnsemble_AllModels_AutoML_20190725_053020,0.700417,0.601023,0.415625,0.454062,0.206172
GBM_grid_1_AutoML_20190725_053020_model_6,0.691458,0.776322,0.472222,0.489209,0.239325


In [14]:
aml.leader

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_grid_1_AutoML_20190725_053020_model_10


ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.02670557027462496
RMSE: 0.1634183902583334
LogLoss: 0.16652576342191194
Mean Per-Class Error: 0.0
AUC: 1.0
pr_auc: 0.9937499999999999
Gini: 1.0
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.7300984263420105: 


,0,1,Error,Rate
0,90.0,0.0,0.0,(0.0/90.0)
1,0.0,160.0,0.0,(0.0/160.0)
Total,90.0,160.0,0.0,(0.0/250.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.7300984,1.0,159.0
max f2,0.7300984,1.0,159.0
max f0point5,0.7300984,1.0,159.0
max accuracy,0.7300984,1.0,159.0
max precision,0.9558320,1.0,0.0
max recall,0.7300984,1.0,159.0
max specificity,0.9558320,1.0,0.0
max absolute_mcc,0.7300984,1.0,159.0
max min_per_class_accuracy,0.7300984,1.0,159.0
max mean_per_class_accuracy,0.7300984,1.0,159.0


Gains/Lift Table: Avg response rate: 64.00 %, avg score: 63.49 %



...

Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
,2019-07-25 05:31:46,24.093 sec,0.0,0.5,0.6931472,0.5,0.0,1.0,0.36
,2019-07-25 05:31:46,24.168 sec,5.0,0.4438398,0.5862145,0.9821528,0.9783660,1.5625,0.068
,2019-07-25 05:31:46,24.241 sec,10.0,0.3955226,0.5013486,0.9945139,0.9907866,1.5625,0.04
,2019-07-25 05:31:46,24.306 sec,15.0,0.3568671,0.4372784,0.9985417,0.9929776,1.5625,0.012
,2019-07-25 05:31:46,24.354 sec,20.0,0.3230448,0.3841441,0.9997917,0.9936346,1.5625,0.004
,2019-07-25 05:31:46,24.394 sec,25.0,0.2948525,0.3419081,1.0,0.9937500,1.5625,0.0
,2019-07-25 05:31:47,24.431 sec,30.0,0.2698593,0.3056458,1.0,0.9937500,1.5625,0.0
,2019-07-25 05:31:47,24.467 sec,35.0,0.2481505,0.2752100,1.0,0.9937500,1.5625,0.0
,2019-07-25 05:31:47,24.502 sec,40.0,0.2278577,0.2478314,1.0,0.9937500,1.5625,0.0
,2019-07-25 05:31:47,24.538 sec,45.0,0.2102973,0.2248279,1.0,0.9937500,1.5625,0.0


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
33,339.1589661,1.0,0.1409011
65,140.3394775,0.4137867,0.0583030
217,120.4042130,0.3550082,0.0500211
117,119.1591034,0.3513370,0.0495038
91,80.4987640,0.2373482,0.0334426
---,---,---,---
233,0.6396351,0.0018859,0.0002657
87,0.4564552,0.0013458,0.0001896
218,0.2399101,0.0007074,0.0000997
1,0.1210165,0.0003568,0.0000503



See the whole table with table.as_data_frame()


In [15]:
preds = aml.predict(test_D)

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [16]:
y_pred = preds['p1'].as_data_frame().values

In [17]:
subm = pd.read_csv('../input/sample_submission.csv')
subm.shape

(19750, 2)

In [18]:
test.shape

(19750, 300)

In [19]:
subm['target'] = y_pred
subm.to_csv('submission.csv', index = False)